In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("trransformation").getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/17 09:18:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
rdd_in = sc.textFile("data/input_real_estate.txt")

In [5]:
rdd_in.collect()

['Property_ID|Location|Price|Bedrooms|Bathrooms|Size|Price_SQ_FT|Status',
 '1461262|Arroyo Grande|795000|3|3|2371|365.3|Short Sale',
 '1478004|Paulo Pablo|399000|4|3|2818|163.59|Short Sale',
 '1486551|Paulo Pablo|545000|4|3|3032|179.75|Short Sale',
 '1492832|Santa Bay|909000|4|4|3540|286.78|Short Sale',
 '1499102|Thomas Country|109900|3|1|1249|98.99|Short Sale',
 '1489132|Thomas Country|109000|2|1|1129|93.99|Short Sale',
 '1467262|Fort Worth|987000|4|3|2771|465.3|Short Sale',
 '1478114|Paulo Pablo|409000|4|3|2918|223.19|Short Sale',
 '1402551|Nashville|545000|4|3|2932|169.75|Short Sale',
 '1405832|San Jose|980000|4|4|3340|290.98|Short Sale',
 '1493302|Fort Worth|119900|3|2|2249|198.99|Short Sale',
 '1412332|Thomas Country|129000|3|2|1329|73.99|Short Sale',
 '1469062|Arroyo Grande|798000|3|4|2321|235.9|Short Sale',
 '1498004|Nashville|789000|4|3|2419|263.59|Short Sale',
 '1586751|Nashville|896000|4|3|3132|199.75|Short Sale',
 '1433232|Glendale|987000|4|4|3340|216.78|Short Sale',
 '14955

In [15]:
# Distinguish header and data
rdd1 = rdd_in.filter(lambda l: not l.startswith("Property_ID"))
header = rdd_in.filter(lambda l: l.startswith("Property_ID"))



In [16]:
header.collect()[0].split("|")

['Property_ID',
 'Location',
 'Price',
 'Bedrooms',
 'Bathrooms',
 'Size',
 'Price_SQ_FT',
 'Status']

In [17]:
# Applying the flatmap and map to get the column data
rdd2 = rdd1.flatMap(lambda x:x.split(",")).map(lambda x:x.split("|"))


In [19]:
# Get the index of required column
col_list = header.first().split('|')
i1 = col_list.index("Property_ID")
i2 = col_list.index("Location")
i3 = col_list.index("Size")
i4 = col_list.index("Price_SQ_FT")




In [20]:
def final_price(d1,d2):
    result = float(d1)*float(d2)
    return str(result)

In [22]:
header_out = header.map(lambda x:x.split("|")[i1]+"|"+x.split('|')[i2]+'|final_price')
header_out.collect()

['Property_ID|Location|final_price']

In [25]:
rdd3 = rdd2.map(lambda x: x[i1]+'|'+x[i2]+'|'+final_price(x[i3],x[i4]))    
rdd3.collect()                                                                           

['1461262|Arroyo Grande|866126.3',
 '1478004|Paulo Pablo|460996.62',
 '1486551|Paulo Pablo|545002.0',
 '1492832|Santa Bay|1015201.2',
 '1499102|Thomas Country|123638.51',
 '1489132|Thomas Country|106114.70999999999',
 '1467262|Fort Worth|1289346.3',
 '1478114|Paulo Pablo|651268.42',
 '1402551|Nashville|497707.0',
 '1405832|San Jose|971873.2000000001',
 '1493302|Fort Worth|447528.51',
 '1412332|Thomas Country|98332.70999999999',
 '1469062|Arroyo Grande|547523.9',
 '1498004|Nashville|637624.21',
 '1586751|Nashville|625617.0',
 '1433232|Glendale|724045.2',
 '1495502|Fort Worth|198377.13',
 '1489100|San Jose|62323.10999999999']

In [27]:
final_out = header_out.union(rdd3)
final_out.collect()

['Property_ID|Location|final_price',
 '1461262|Arroyo Grande|866126.3',
 '1478004|Paulo Pablo|460996.62',
 '1486551|Paulo Pablo|545002.0',
 '1492832|Santa Bay|1015201.2',
 '1499102|Thomas Country|123638.51',
 '1489132|Thomas Country|106114.70999999999',
 '1467262|Fort Worth|1289346.3',
 '1478114|Paulo Pablo|651268.42',
 '1402551|Nashville|497707.0',
 '1405832|San Jose|971873.2000000001',
 '1493302|Fort Worth|447528.51',
 '1412332|Thomas Country|98332.70999999999',
 '1469062|Arroyo Grande|547523.9',
 '1498004|Nashville|637624.21',
 '1586751|Nashville|625617.0',
 '1433232|Glendale|724045.2',
 '1495502|Fort Worth|198377.13',
 '1489100|San Jose|62323.10999999999']

In [29]:
final_out.coalesce(1).saveAsTextFile("data/output.txt")

In [30]:
final_out.collect()

['Property_ID|Location|final_price',
 '1461262|Arroyo Grande|866126.3',
 '1478004|Paulo Pablo|460996.62',
 '1486551|Paulo Pablo|545002.0',
 '1492832|Santa Bay|1015201.2',
 '1499102|Thomas Country|123638.51',
 '1489132|Thomas Country|106114.70999999999',
 '1467262|Fort Worth|1289346.3',
 '1478114|Paulo Pablo|651268.42',
 '1402551|Nashville|497707.0',
 '1405832|San Jose|971873.2000000001',
 '1493302|Fort Worth|447528.51',
 '1412332|Thomas Country|98332.70999999999',
 '1469062|Arroyo Grande|547523.9',
 '1498004|Nashville|637624.21',
 '1586751|Nashville|625617.0',
 '1433232|Glendale|724045.2',
 '1495502|Fort Worth|198377.13',
 '1489100|San Jose|62323.10999999999']